<a href="https://colab.research.google.com/github/doumsdd/PnamaSQL/blob/main/PanamapapersSQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import sqlite3

# Establish a connection to the SQLite database
conn = sqlite3.connect("/content/panamapapers.sqlite3")

# Create a cursor object
cur = conn.cursor()

# Execute a query to get the list of all tables in the database
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Fetch the result
tables = cur.fetchall()

# Close the cursor and the connection
cur.close()
conn.close()

# The query returns a list of tuples. We will transform this into a simple list of table names.
table_names = [table[0] for table in tables]

table_names


['source',
 'status',
 'country',
 'address',
 'intermediary',
 'entity',
 'officer',
 'assoc_officer_entity',
 'assoc_inter_entity',
 'assoc_officers',
 'assoc_intermediaries',
 'assoc_entities',
 'assoc_inter_offi',
 'assoc_officer_interm']

In [3]:
# Pour répondre à la première question, nous devons probablement examiner la table 'entity' et la table 'source'.
# Cependant, nous avons besoin de plus d'informations sur la structure de ces tables. Allons voir les schémas de ces tables.
# Create a function to get the schema of a table
def get_table_schema(conn, table_name):
    cur = conn.cursor()
    cur.execute(f"PRAGMA table_info({table_name});")
    schema = cur.fetchall()
    cur.close()
    return schema

# Establish a connection to the SQLite database
conn = sqlite3.connect("/content/panamapapers.sqlite3")

# Get the schema of the 'entity' and 'source' tables
entity_schema = get_table_schema(conn, 'entity')
source_schema = get_table_schema(conn, 'source')

conn.close()

(entity_schema, source_schema)

([(0, 'id', 'INTEGER', 0, None, 1),
  (1, 'name', 'TEXT', 0, None, 0),
  (2, 'jurisdiction', 'TEXT', 0, None, 0),
  (3, 'jurisdiction_description', 'TEXT', 0, None, 0),
  (4, 'incorporation_date', 'DATE', 0, None, 0),
  (5, 'status', 'TEXT', 0, None, 0),
  (6, 'service_provider', 'TEXT', 0, None, 0),
  (7, 'source', 'TEXT', 0, None, 0),
  (8, 'note', 'TEXT', 0, None, 0),
  (9, 'id_address', 'BIGINT', 0, None, 0),
  (10, 'end_date', 'DATE', 0, None, 0),
  (11, 'url', 'TEXT', 0, None, 0),
  (12, 'lifetime', 'INTEGER', 0, None, 0)],
 [(0, 'id', 'INTEGER', 0, None, 1), (1, 'source', 'TEXT', 0, None, 0)])

In [4]:
#Pour répondre à la première question ("Combien la base de données contient-elle de sociétés offshores différentes dont la source est 'Panama Papers'?"),
#nous devons compter le nombre de lignes uniques dans la table 'entity' où la colonne 'source' est égale à 'Panama Papers'.

# Establish a connection to the SQLite database
conn = sqlite3.connect("/content/panamapapers.sqlite3")

# Create a cursor object
cur = conn.cursor()

# Execute a query to count the number of unique entities where source is 'Panama Papers'
cur.execute("SELECT COUNT(DISTINCT id) FROM entity WHERE source = 'Panama Papers';")

# Fetch the result
num_entities_panama_papers = cur.fetchone()[0]

# Close the cursor and the connection
cur.close()
conn.close()

num_entities_panama_papers


213634

Pour répondre à la deuxième question ("Quel intermédiaire a créé le plus de sociétés offshores ? A-t-on son adresse et son pays ?"), nous devons faire une jointure entre ces deux tables sur l'identifiant de l'intermédiaire, puis compter le nombre d'entités associées à chaque intermédiaire. Ensuite, nous trierons les résultats par ordre décroissant et prendrons le premier.

Exécutons cette requête maintenant. Nous devons également joindre la table 'address' pour obtenir l'adresse de l'intermédiaire. Obtenons d'abord le schéma de la table 'address'.

In [5]:
# Establish a connection to the SQLite database
conn = sqlite3.connect("/content/panamapapers.sqlite3")

# Get the schema of the 'address' table
address_schema = get_table_schema(conn, 'address')

conn.close()

address_schema


[(0, 'address', 'TEXT', 0, None, 0),
 (1, 'countries', 'TEXT', 0, None, 0),
 (2, 'country_codes', 'TEXT', 0, None, 0),
 (3, 'id_address', 'BIGINT', 0, None, 1),
 (4, 'source_id', 'INTEGER', 0, None, 0)]

La table 'address' a les colonnes suivantes :

'address'
'countries'
'country_codes'
'id_address'
'source_id'
Maintenant, joignons les trois tables 'intermediary', 'assoc_inter_entity', et 'address' pour trouver l'intermédiaire qui a créé le plus de sociétés offshores, ainsi que son adresse et son pays.

In [6]:
# Establish a connection to the SQLite database
conn = sqlite3.connect("/content/panamapapers.sqlite3")

# Create a cursor object
cur = conn.cursor()

# Execute a query to find the intermediary who created the most offshore companies
cur.execute("""
    SELECT
        intermediary.name,
        COUNT(assoc_inter_entity.entity) AS num_entities,
        address.address,
        address.countries
    FROM intermediary
    INNER JOIN assoc_inter_entity ON intermediary.id = assoc_inter_entity.inter
    LEFT JOIN address ON intermediary.id_address = address.id_address
    GROUP BY intermediary.id
    ORDER BY num_entities DESC
    LIMIT 1;
""")

# Fetch the result
top_intermediary = cur.fetchone()

# Close the cursor and the connection
cur.close()
conn.close()

top_intermediary


('ORION HOUSE SERVICES (HK) LIMITED',
 7016,
 'ORION HOUSE SERVICES (HK) LIMITED ROOM 1401; 14/F.; WORLD COMMERCE  CENTRE;[...] TSUI; KOWLOON; HONG KONG',
 'HKG')

Passons maintenant à la troisième question.

Combien la base contient-elle de bénéficiaires avec un nom unique ? Quel est le bénéficiaire dont le nom revient le plus souvent ?
Pour répondre à cette question, nous devons examiner la table 'officer'. Commençons par explorer le schéma de cette table.

In [7]:
# Establish a connection to the SQLite database
conn = sqlite3.connect("/content/panamapapers.sqlite3")

# Get the schema of the 'officer' table
officer_schema = get_table_schema(conn, 'officer')

conn.close()

officer_schema



[(0, 'id', 'INTEGER', 0, None, 1),
 (1, 'name', 'TEXT', 0, None, 0),
 (2, 'source_id', 'INTEGER', 0, None, 0),
 (3, 'note', 'TEXT', 0, None, 0),
 (4, 'country_id', 'INTEGER', 0, None, 0)]

La table 'officer' a les colonnes suivantes :

'id'
'name'
'source_id'
'note'
'country_id'

Pour répondre à la troisième question ("Combien la base contient-elle de bénéficiaires avec un nom unique ? Quel est le bénéficiaire dont le nom revient le plus souvent ?"), nous devons compter le nombre de noms uniques dans la table 'officer' et trouver le nom qui revient le plus souvent.

Exécutons ces deux requêtes maintenant.

In [8]:
# Establish a connection to the SQLite database
conn = sqlite3.connect("/content/panamapapers.sqlite3")

# Create a cursor object
cur = conn.cursor()

# Execute a query to count the number of unique officer names
cur.execute("SELECT COUNT(DISTINCT name) FROM officer;")

# Fetch the result
num_unique_officers = cur.fetchone()[0]

# Execute a query to find the officer name that occurs the most frequently
cur.execute("""
    SELECT name, COUNT(*) AS num_occurrences
    FROM officer
    GROUP BY name
    ORDER BY num_occurrences DESC
    LIMIT 1;
""")

# Fetch the result
most_common_officer = cur.fetchone()

# Close the cursor and the connection
cur.close()
conn.close()

(num_unique_officers, most_common_officer)


(122590, ('THE BEARER', 70850))

La base de données contient 122 590 bénéficiaires avec un nom unique.

Le bénéficiaire dont le nom revient le plus souvent est "THE BEARER" qui apparaît 70 850 fois.

Passons maintenant à la quatrième question.

Donner la liste des juridictions avec le nombre d'entreprises offshores enregistrées sur chaque territoire, triée par ordre décroissant.
Pour répondre à cette question, nous devons regrouper les données de la table 'entity' par juridiction et compter le nombre d'entités dans chaque juridiction. Exécutons cette requête maintenant.

Voici la liste des juridictions avec le nombre d'entreprises offshores enregistrées sur chaque territoire, triée par ordre décroissant :

In [9]:
# Establish a connection to the SQLite database
conn = sqlite3.connect("/content/panamapapers.sqlite3")

# Create a cursor object
cur = conn.cursor()

# Execute a query to get the list of jurisdictions with the number of offshore companies registered in each jurisdiction
cur.execute("""
    SELECT jurisdiction, COUNT(*) AS num_entities
    FROM entity
    GROUP BY jurisdiction
    ORDER BY num_entities DESC;
""")

# Fetch the result
jurisdictions = cur.fetchall()

# Close the cursor and the connection
cur.close()
conn.close()

jurisdictions


[('BVI', 114849),
 ('PMA', 48870),
 ('BAH', 16082),
 ('SEY', 15341),
 ('NIUE', 9711),
 ('SAM', 5361),
 ('ANG', 3286),
 ('NEV', 1273),
 ('HK', 457),
 ('UK', 151),
 ('BLZ', 133),
 ('CRI', 81),
 ('CYP', 78),
 ('UY', 54),
 ('NZL', 49),
 ('JSY', 41),
 ('WYO', 39),
 ('MLT', 29),
 ('IOM', 8),
 ('RAK', 2),
 ('SGP', 1)]

Veuillez noter que les codes des juridictions sont probablement des codes ISO à 3 lettres.

Passons maintenant à la cinquième question.

Regrouper les sociétés offshores par statut, et trier la liste par ordre décroissant.
Pour répondre à cette question, nous devons regrouper les données de la table 'entity' par statut et compter le nombre d'entités pour chaque statut. Exécutons cette requête maintenant.

In [10]:
# Establish a connection to the SQLite database
conn = sqlite3.connect("/content/panamapapers.sqlite3")

# Create a cursor object
cur = conn.cursor()

# Execute a query to group the offshore companies by status and sort the list in descending order
cur.execute("""
    SELECT status, COUNT(*) AS num_entities
    FROM entity
    GROUP BY status
    ORDER BY num_entities DESC;
""")

# Fetch the result
statuses = cur.fetchall()

# Close the cursor and the connection
cur.close()
conn.close()

statuses


[('Defaulted', 100090),
 ('Active', 57990),
 ('Dissolved', 22377),
 ('Changed agent', 16043),
 ('Inactivated', 7640),
 ('Resigned as agent', 3174),
 ('Shelf company', 2576),
 ('Dissolved shelf company', 1573),
 ('Bad debt account', 1416),
 ('Trash company', 878),
 ('In transition', 776),
 ('Relocated in new jurisdiction', 672),
 ('Discontinued', 423),
 ('Shelf company not possible to sell', 201),
 ('In liquidation', 40),
 ('Change in administration pending', 21),
 ('Resigned as agent of shelf company', 3),
 (None, 3)]

Voici la liste des statuts des sociétés offshores, triée par ordre décroissant du nombre d'entités :



Trouver la liste des bénéficiaires dont le nom contient "BNP" et ajouter, pour chaque bénéficiaire, le nom des sociétés offshores. Le tout en requête SQL.
Pour répondre à cette question, nous devons faire une jointure entre les tables 'officer', 'assoc_officer_entity' et 'entity', puis filtrer les résultats pour les noms d'officiers contenant "BNP". Exécutons cette requête maintenant.

In [11]:
# Establish a connection to the SQLite database
conn = sqlite3.connect("/content/panamapapers.sqlite3")

# Create a cursor object
cur = conn.cursor()

# Execute a query to find the list of officers whose name contains "BNP" and add the name of the offshore companies for each officer
cur.execute("""
    SELECT officer.name AS officer_name, entity.name AS entity_name
    FROM officer
    INNER JOIN assoc_officer_entity ON officer.id = assoc_officer_entity.officer
    INNER JOIN entity ON assoc_officer_entity.entity = entity.id
    WHERE officer.name LIKE '%BNP%'
    ORDER BY officer.name;
""")

# Fetch the result
officers_with_bnp = cur.fetchall()

# Close the cursor and the connection
cur.close()
conn.close()

officers_with_bnp


[('BGL BNP PARIBAS SOCIETE ANONYME IN THE NOMINEE OF FORTIS LUXEMBOURG VI',
  'DIARIO ASSETS S.A.'),
 ('BNP JERSEY NOMINEE CO. LTD', 'KARLINGTON LIMITED'),
 ('BNP JERSEY NOMINEE CO. LTD', 'TATRON LIMITED'),
 ('BNP JERSEY NOMINEE COMPANY', 'THOR INVESTMENTS LIMITED'),
 ('BNP JERSEY NOMINEE COMPANY LIMITED', 'TURF OVERSEAS LIMITED'),
 ('BNP JERSEY NOMINEE COMPANY LIMITED', 'TEMPLEMEAD LIMITED'),
 ('BNP JERSEY TRUST CORPORATION LIMITED', 'KARLINGTON LIMITED'),
 ('BNP JERSEY TRUST CORPORATION LIMITED', 'TATRON LIMITED'),
 ('BNP Jersey Nominee Company Limited', 'SAKI FINANCE LIMITED'),
 ('BNP Jersey Nominees Company', "SALLY'S SUNSHINE LTD."),
 ('BNP Jersey Trust Corporation Limited', 'SAKI FINANCE LIMITED'),
 ('BNP PARIBAS JERSEY NOMINE COMPANY LIMITED', 'MOLDESTAD HOLDING LIMITED'),
 ('BNP PARIBAS JERSEY NOMINEE COMPANY LIMITED', 'MAJESTIC FINANCE CORP.'),
 ('BNP PARIBAS JERSEY NOMINEE COMPANY LIMITED',
  'DRAGON STRATEGIC INVESTMENT INC.'),
 ('BNP PARIBAS JERSEY NOMINEE COMPANY LIMITED',

Voici la liste des bénéficiaires dont le nom contient "BNP", ainsi que le nom des sociétés offshores pour chaque bénéficiaire :